# Final Notebook - Rap Generator

Nowadays, mainstream music has become so formulaic that a computer could probably write it. I decided to put that to the test by creating a program to generate song lyrics for a given artist.

## Strategy
1. First, we will fetch collections of lyrics for a given artist.
2. Next, generate an n-gram model for that artist.
3. Define a stress and rhyme pattern for a song.
4. Repeatedly generate n-grams, accepting only those that match the above pattern, until song is complete.

In [9]:
# Import our API keys
from utility.config import *

# We will use this package to make API calls
import requests
import urllib
from lyricsgenius import Genius

genius = Genius(ACCESS_TOKEN)

HEADERS = {'User-Agent': 'XY', 'Content-type': 'application/json', 'Authorization': 'Bearer ' + ACCESS_TOKEN}
API_URL = 'https://api.genius.com/'

First, let's decide which artist we are going to use. The following cell searches for a given artist with Genius's API. Enter in the artist for whom you'd like to generate lyrics–it will work better if they have a large discography.

In [10]:
artist_found = False
artist_id = ''
artist_name = ''

while not artist_found:
    print('What artist would you like to search for?')
    query = input().strip()
    
    # encode spaces
    params = urllib.parse.urlencode({'q': query}, quote_via=urllib.parse.quote)
    
    resp = requests.get(API_URL + 'search', headers=HEADERS, params=params).json()
    
    if len(resp['response']['hits']) == 0:
        print('No hits found')
        continue
    
    for hit in resp['response']['hits']:
        if hit['type'] == 'song':
            if hit['result']['primary_artist']['name'].lower() == query.lower():
                print('Did ' + query + ' sing \'' + hit['result']['title'] + '\'? Enter y for yes.')
                confirm = input()
                if confirm.strip() == 'y':
                    # We have found the right artist! Let's get out of here.
                    artist_id = hit['result']['primary_artist']['id']
                    artist_name = hit['result']['primary_artist']['name']
                    artist_found = True
                    break
    
print(artist_name)
print(artist_id)

What artist would you like to search for?


 Eminem


{'meta': {'status': 200}, 'response': {'hits': [{'highlights': [], 'index': 'song', 'type': 'song', 'result': {'annotation_count': 110, 'api_path': '/songs/235729', 'full_title': 'Rap God by\xa0Eminem', 'header_image_thumbnail_url': 'https://images.genius.com/058e2359838c93395c36119b48a2eff6.300x300x1.png', 'header_image_url': 'https://images.genius.com/058e2359838c93395c36119b48a2eff6.1000x1000x1.png', 'id': 235729, 'lyrics_owner_id': 22533, 'lyrics_state': 'complete', 'path': '/Eminem-rap-god-lyrics', 'pyongs_count': 6504, 'song_art_image_thumbnail_url': 'https://images.genius.com/058e2359838c93395c36119b48a2eff6.300x300x1.png', 'song_art_image_url': 'https://images.genius.com/058e2359838c93395c36119b48a2eff6.1000x1000x1.png', 'stats': {'unreviewed_annotations': 0, 'concurrents': 10, 'hot': False, 'pageviews': 17015201}, 'title': 'Rap God', 'title_with_featured': 'Rap God', 'url': 'https://genius.com/Eminem-rap-god-lyrics', 'song_art_primary_color': '#dd4448', 'song_art_secondary_col

KeyboardInterrupt: Interrupted by user

Now, we must fetch a reasonable amount of songs for the artist. We will repeatedly query the Genius API for songs for the artist. We might also want to eliminate songs with features on them, since that will muddy our n-gram grammar. Note: this step should take quite a bit, as it has to make many API calls.

In [39]:
artist_songs = []

more_to_get = True
cur_page = 1 # Used to page through all songs

while more_to_get:
    params = {'sort': 'popularity', 'page': cur_page}
    resp = requests.get(API_URL + 'artists/' + str(artist_id) + '/songs', headers=HEADERS, params=params).json()
        
    if len(resp['response']['songs']) == 0:
        more_to_get = False
        break
        
    for song in resp['response']['songs']:
        # Let's make sure the song matches a few basic conditions
        # First: is the main artist correct?
        if not song['primary_artist']['id'] == artist_id:
            break
        
        # Next: is there a feature?
        if 'feat.' in song['title'].lower() or 'ft.' in song['title'].lower():
            break
            
        # If neither, we are good to go.
        artist_songs.append(song)
        print(song['title'])
    
    cur_page += 1
    
    if len(artist_songs) > 500:
        more_to_get = False
        break

print('Done. Got ' + str(len(artist_songs)) + ' songs.')

FEFE
GOOBA
GUMMO
BILLY
KOODA
TATI
YOKAI
CHOCOLATÉ
ScumLife
Zeta Zero 0.5
On The Regular
BUBA
GTL
SINALØA
Shinigami (死神)
NINI
Fuck Is You
LEAH
CHARLIE
CRIMINAL*
Alright
Lose Yourself (Freestyle)
PUNANI (Demo)
NADA*
SCUM
Body Buildings
International Gangstas (Original Version)
Done. Got 27 songs.


Now, we will use these songs to generate a single n-gram model for the artist.

In [40]:
artist_songs

[{'annotation_count': 23,
  'api_path': '/songs/3767821',
  'full_title': 'FEFE by\xa06ix9ine (Ft.\xa0Murda\xa0Beatz & Nicki\xa0Minaj)',
  'header_image_thumbnail_url': 'https://images.genius.com/455ffbf3409022786dc15470da6d93e9.300x160x1.png',
  'header_image_url': 'https://images.genius.com/455ffbf3409022786dc15470da6d93e9.806x430x1.png',
  'id': 3767821,
  'lyrics_owner_id': 1395662,
  'lyrics_state': 'complete',
  'path': '/6ix9ine-fefe-lyrics',
  'pyongs_count': 250,
  'song_art_image_thumbnail_url': 'https://images.genius.com/986690d69bc839048896aa1d10d46356.300x300x1.png',
  'song_art_image_url': 'https://images.genius.com/986690d69bc839048896aa1d10d46356.640x640x1.png',
  'stats': {'unreviewed_annotations': 1,
   'concurrents': 2,
   'hot': False,
   'pageviews': 4342594},
  'title': 'FEFE',
  'title_with_featured': 'FEFE (Ft.\xa0Murda\xa0Beatz & Nicki\xa0Minaj)',
  'url': 'https://genius.com/6ix9ine-fefe-lyrics',
  'song_art_primary_color': '#14d3fb',
  'song_art_secondary_col